# FlavorFinder

Aplikacja pozwala przeglądać dostępne dania z darmowego api [mealAPI](https://www.themealdb.com), oraz dodać ulubione do lokalnej bazy danych.

Aplikacja wykorzystuje bazę danych `ROOM` do przechowywania ulubionych dań, oraz bibliotekę `Retrofit` do pobrania danych z zewnętrznego serwera.

<table><tr><td><img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExZzlweXR6dzI1b3ZpNnk4MXo3cTJ6Yjd1NWx3ZG85ZGNzNGkyOHZobCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/NqBQ91k7zOVh5d39NK/giphy.gif" width="200" /></td><td><img src="https://media3.giphy.com/media/v1.Y2lkPTc5MGI3NjExY29iNXd2Z2hiYnhldWtpeHVpeTR5YmYwM3ltZWh3bHlyYmkwcTNneCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/WiFtop3Zg3tdTUGFnh/giphy.gif" width="200" /></td><td><img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExOGs3ZTZmMnpsb21pdTAwbWphbGo1MmxoZGFtNmUwaGlzdmttcndwcCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/Van1cMPDSfS9haqhIL/giphy.gif" width="200" /></td></tr></table>

- Aplikacja zaimplementowana zgodnie ze wzoccem MVVM wraz z repozytorium
- Repozytorium zawiera metody dostępowe do lokalnej bazy oraz zewnętrznego api
- biblioteka `Retrofit` została wykorzystana do połączenia z zewnętrznym serwerem
- biblioteka `Coil` została wykorzystana do obsługi ładowania grafik
- `StateFlow` został wykorzystany do łatwego zarządzania bieżącym stanem interfejsu użytkownika
- Baza danych `Room` została wykorzystana do przechowywania dań
- Aplikacja zawiera trzy ekrany 
    - na głównym ekranie wyświetla listę dań pobranych z serwera
    - ekran *ulubione* pozwala przeglądać dania zapisane w lokalnej bazy danych
    - trzeci ekran umożliwia uzyskanie większej ilości informacji o wybranym daniu
- Aplikacja wykorzystuje `Compose Navigation` w celu utworzenia nawigacji w aplikacji
- `ViewModel` został umieszczony w komponencie `Navigation` w celu uzyskania współdzielenia jego instancji przez wszystkie ekrany


Dodajmy wymagane zależności do projektu

```kotlin
plugins {
    id("com.android.application")
    id("org.jetbrains.kotlin.android")
    id ("kotlin-kapt")
}

dependencies {
    implementation ("androidx.lifecycle:lifecycle-viewmodel-compose:2.6.2")
    implementation ("androidx.lifecycle:lifecycle-viewmodel-ktx:2.6.2")
    implementation ("androidx.lifecycle:lifecycle-runtime-compose:2.6.2")

    implementation ("com.squareup.retrofit2:retrofit:2.9.0")
    implementation ("com.squareup.retrofit2:converter-gson:2.9.0")
    implementation ("com.squareup.okhttp3:logging-interceptor:4.10.0")

    implementation ("androidx.room:room-runtime:2.5.2")
    annotationProcessor ("androidx.room:room-compiler:2.5.2")
    kapt ("androidx.room:room-compiler:2.5.2")
    implementation("androidx.room:room-ktx:2.5.2")

    implementation ("androidx.navigation:navigation-compose:2.6.0")

    implementation ("androidx.compose.material:material-icons-extended:1.4.3")

    implementation("io.coil-kt:coil:2.4.0")
    implementation("io.coil-kt:coil-compose:2.4.0")
    ...
}
```

dodajmy również odpowiednie upoważnienia do manifestu

```xml
<uses-permission android:name="android.permission.INTERNET"/>
```

Przejrzyjmy dane które wykorzystamy z api. W tej aplikacji pobierzemy tylko polskie dania, korzystając z endpointu https://www.themealdb.com/api/json/v1/1/filter.php?a=Polish. Zobaczmy odpowiedź.

In [ ]:
{
   "meals":[
      {
         "strMeal":"Bigos (Hunters Stew)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/md8w601593348504.jpg",
         "idMeal":"53018"
      },
      {
         "strMeal":"Go\u0142\u0105bki (cabbage roll)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/q8sp3j1593349686.jpg",
         "idMeal":"53021"
      },
      {
         "strMeal":"Paszteciki (Polish Pasties)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/c9a3l31593261890.jpg",
         "idMeal":"53017"
      },
      {
         "strMeal":"Pierogi (Polish Dumplings)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/45xxr21593348847.jpg",
         "idMeal":"53019"
      },
      {
         "strMeal":"Polskie Nale\u015bniki (Polish Pancakes)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/58bkyo1593350017.jpg",
         "idMeal":"53022"
      },
      {
         "strMeal":"Rogaliki (Polish Croissant Cookies)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/7mxnzz1593350801.jpg",
         "idMeal":"53024"
      },
      {
         "strMeal":"Ros\u00f3\u0142 (Polish Chicken Soup)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/lx1kkj1593349302.jpg",
         "idMeal":"53020"
      },
      {
         "strMeal":"Sledz w Oleju (Polish Herrings)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/7ttta31593350374.jpg",
         "idMeal":"53023"
      }
   ]
}

Dostaniemy nazwę, link do zdjęcia oraz identyfikator. Chcemy również dostać więcej informacji o wybranym przez użytkownika daniu, w tym celu wykorzystamy endpoint https://www.themealdb.com/api/json/v1/1/lookup.php?i=52772. Po podaniu identyfikatora otrzymamy następujące dane.

In [ ]:
{
   "meals":[
      {
         "idMeal":"52772",
         "strMeal":"Teriyaki Chicken Casserole",
         "strDrinkAlternate":null,
         "strCategory":"Chicken",
         "strArea":"Japanese",
         "strInstructions":"Preheat oven to 350\u00b0 F. Spray a 9x13-inch baking pan with non-stick spray.\r\nCombine soy sauce, ...   
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/wvpsxx1468256321.jpg",
         "strTags":"Meat,Casserole",
         "strYoutube":"https:\/\/www.youtube.com\/watch?v=4aZr5hZXP_s",
         "strIngredient1":"soy sauce",
         "strIngredient2":"water",
         "strIngredient3":"brown sugar",
         "strIngredient4":"ground ginger",
         "strIngredient5":"minced garlic",
         "strIngredient6":"cornstarch",
         "strIngredient7":"chicken breasts",
         "strIngredient8":"stir-fry vegetables",
         "strIngredient9":"brown rice",
         "strIngredient10":"",
         "strIngredient11":"",
         "strIngredient12":"",
         "strIngredient13":"",
         "strIngredient14":"",
         "strIngredient15":"",
         "strIngredient16":null,
         "strIngredient17":null,
         "strIngredient18":null,
         "strIngredient19":null,
         "strIngredient20":null,
         "strMeasure1":"3\/4 cup",
         "strMeasure2":"1\/2 cup",
         "strMeasure3":"1\/4 cup",
         "strMeasure4":"1\/2 teaspoon",
         "strMeasure5":"1\/2 teaspoon",
         "strMeasure6":"4 Tablespoons",
         "strMeasure7":"2",
         "strMeasure8":"1 (12 oz.)",
         "strMeasure9":"3 cups",
         "strMeasure10":"",
         "strMeasure11":"",
         "strMeasure12":"",
         "strMeasure13":"",
         "strMeasure14":"",
         "strMeasure15":"",
         "strMeasure16":null,
         "strMeasure17":null,
         "strMeasure18":null,
         "strMeasure19":null,
         "strMeasure20":null,
         "strSource":null,
         "strImageSource":null,
         "strCreativeCommonsConfirmed":null,
         "dateModified":null
      }
   ]
}

## Data

Wybierzmy tylko pola, które będą nas interesować i stwórzmy model. Dane dostaniemy w obu przypadkach jako listę, więc dodajmy klasę `MealRepsonse` zawierającą listę obiektów typu `Meal`. W klasie `Meal` definiujemy tylko pola, które nas interesują. Dla obu odpowiedzi możemy posłużyć się jednym modelem (nie ma konieczności tworzenia wielu modeli, jeżeli przynajmniej część danych się powtarza)

In [ ]:
data class MealResponse(
    val meals: List<Meal>
)

In [ ]:
@Entity(tableName = "meal")
data class Meal(
    @PrimaryKey
    val idMeal: String,
    val strArea: String,
    val strCategory: String,
    val strInstructions: String,
    val strMeal: String,
    val strMealThumb: String,
)

Dodajmy interfejs reprezentujący api, i zdefiniujmy w nim dwie metody, które zwracają wszystkie dania, oraz danie o zadanym id.

In [ ]:
interface MealApi {
    @GET("api/json/v1/1/filter.php?a=Polish")
    suspend fun getFood() : Response<MealResponse>

    @GET("api/json/v1/1/lookup.php?")
    suspend fun getFoodById(@Query("i") id: String) : Response<MealResponse>
}

Następnie dodajmy obiekt reprezentujący instancję `Retrofit`

In [ ]:
object RetrofitInstance {
    val api: MealApi by lazy {
        Retrofit.Builder()
            .baseUrl("https://www.themealdb.com/")
            .addConverterFactory(GsonConverterFactory.create())
            .build()
            .create(MealApi::class.java)
    }
}

Dodajmy `Dao` dla bazy `ROOM` ze zdefiniowanymi metodami dla dodawania, usuwania, oraz zwrócenia wszystkich elementów lokalnej bazy

In [ ]:
@Dao
interface MealDao {
    @Insert(onConflict = REPLACE)
    suspend fun insert(meal: Meal)

    @Delete
    suspend fun delete(meal: Meal)

    @Query("SELECT * FROM meal")
    fun getAllMeals() : Flow<List<Meal>>
}

Następnie dodajmy klasę abstrakcyjną reprezentującą naszą bazę danych.

In [ ]:
@Database(entities = [Meal::class], version = 1, exportSchema = false)
abstract class MealDatabase : RoomDatabase() {
    abstract fun mealDao(): MealDao

    companion object{
        @Volatile private var INSTANCE: MealDatabase? = null

        fun getDatabase(context: Context): MealDatabase {
            return INSTANCE ?: synchronized(this){
                val instance = Room.databaseBuilder(
                    context.applicationContext,
                    MealDatabase::class.java,
                    "meal_database_compose_v3"
                ).build().also { INSTANCE = it }
                instance
            }
        }
    }
}

Dodajmy repozytorium z metodami dostępowymi.

In [ ]:
class MealRepository(application: Application) {

    private val mealDao = MealDatabase.getDatabase(application).mealDao()

    suspend fun fetchData() = RetrofitInstance.api.getFood() // pobranie listy z serwera
    suspend fun fetchById(id: String) = RetrofitInstance.api.getFoodById(id) // pobranie danych o daniu o zadanym id

    val readData: Flow<List<Meal>> = mealDao.getAllMeals() // pobranie listy z lokalnej bazy
    suspend fun insert(meal: Meal) = mealDao.insert(meal) // dodanie elementu do lokalnej bazy
    suspend fun delete(meal: Meal) = mealDao.delete(meal) // usunięcie elementu z lokalnej bazy
}

## ViewModel

Wykorzystamy również wzorzec `ResourceBound` w celu ułatwienia reakcji na różne stany.

In [ ]:
sealed class Resource<T> (
    val data: T? = null,
    val message: String? = null
){
    class Success<T>(data: T) : Resource<T>(data)
    class Error<T>(message: String, data: T? = null) : Resource<T>(data, message)
    class Loading<T> : Resource<T>()
}

Ponieważ `ROOM` wymaga podania kontekstu, musimy dodać implementację fabryki dla `ViewModel`

In [ ]:
class FoodViewModelFactory(private val application: Application) :
    ViewModelProvider.Factory {
    override fun <T : ViewModel> create(modelClass: Class<T>): T {
        return FoodViewModel(application) as T
    }
}

Rozpocznijmy implementację `ViewModel` od dodania pól

In [ ]:
class FoodViewModel(application: Application) : ViewModel() {

    private val repository = MealRepository(application)

    private var _meals: MutableStateFlow<Resource<MealResponse>> = MutableStateFlow(Resource.Loading())
    val meals: StateFlow<Resource<MealResponse>> = _meals

    private var _meal: MutableStateFlow<Resource<MealResponse>> = MutableStateFlow(Resource.Loading())
    val meal: StateFlow<Resource<MealResponse>> = _meal

    val localMeals: StateFlow<List<Meal>> = repository.readData.stateIn(
        viewModelScope,
        SharingStarted.WhileSubscribed(),
        emptyList()
    )
    ...
}

- `meals` przechowuje listę wszystkich dań pobranych z serwera, inicjujemy stanem ładowania
- `meal` przechowuje informację o wybranym daniu pobraną z serwera, inicjujemy stanem ładowania
- `localMeals` przechowuje listę wszystkich dań dodanych do listy ulubionych i zapisanych w lokalnej bazie danych

Dodajmy metodę pomocniczą do obsługi odpowiedzi z serwera.

In [ ]:
    private fun handleMealResponse(response: Response<MealResponse>)
            : Resource<MealResponse> {
        if (response.isSuccessful)
            response.body()?.let { return Resource.Success(it) }
        return Resource.Error(response.message())
    }

- `if (response.isSuccessful)`: Ten fragment kodu sprawdza, czy odpowiedź HTTP jest udana. 
- `response.body()?.let { return Resource.Success(it) }`: Jeśli odpowiedź jest udana, to następnie używana jest funkcja `let`, która wywołuje blok kodu w przypadku, gdy `response.body()` nie jest `null`. To jest bezpieczny sposób dostępu do ciała odpowiedzi. Wewnątrz bloku tworzony jest obiekt `Resource.Success`, który zawiera ciało odpowiedzi. Ostatecznie ten obiekt jest zwracany z funkcji.
- `return Resource.Error(response.message())`: Jeśli odpowiedź nie jest udana, funkcja tworzy obiekt` Resource.Error`, który zawiera komunikat błędu uzyskany z `response.message()`. 

Następnie ten obiekt jest zwracany z funkcji.

W kolejnym kroku zaimplementujmy metody `fetchData` oraz `fetchDataById` pobierające dane z serwera.

In [ ]:
    private fun fetchData() = viewModelScope.launch {
        val response = repository.fetchData()
        delay(2000L) // wykorzystany dla pokazania stanu ładowania
        _meals.value = handleMealResponse(response)
    }
    
    fun fetchById(id: String) = viewModelScope.launch {
        val response = repository.fetchById(id)
        delay(2000L) // wykorzystany dla pokazania stanu ładowania
        _meal.value = handleMealResponse(response)
    }

Wywołujemy odpowiednią funkcję z repozytorium, następnie wykorzystujemy `delay` ab6y przedłużyć stan ładowania o 2 sekundy (pokażemy `CircularProgressBar` podczas ładowania). Ostanim krokiem w tej funkcji jest wywołanie `handleMealResponse`, która zwraca odpowiedni stan (sukces z danymi, lub błąd z odpowiednią wiadomością).

Dodajmy dwie metody obsługujące dodanie i usunięcie elementu z lokalnej bazy.

In [ ]:
    fun insert(meal: Meal) = viewModelScope.launch {
        repository.insert(meal)
    }

    fun delete(meal: Meal) = viewModelScope.launch {
        repository.delete(meal)
    }

Na koniec dodajmy jeszcze metodę sprawdzającą czy element znajduje się na liście przechowującej dane z lokalnej bazy (dzięki temu dodamy odpowiednią funkcjonalność do przycisku odpowiadającego za dodania/usunięcie elementu)

In [ ]:
    fun checkIfExistInLocalDb(meal: Meal): Boolean = meal in localMeals.value

Pełny kod `ViewModel`

In [ ]:
class FoodViewModel(application: Application) : ViewModel() {

    private val repository = MealRepository(application)

    private var _meals: MutableStateFlow<Resource<MealResponse>> = MutableStateFlow(Resource.Loading())
    val meals: StateFlow<Resource<MealResponse>> = _meals

    private var _meal: MutableStateFlow<Resource<MealResponse>> = MutableStateFlow(Resource.Loading())
    val meal: StateFlow<Resource<MealResponse>> = _meal

    val localMeals: StateFlow<List<Meal>> = repository.readData.stateIn(
        viewModelScope,
        SharingStarted.WhileSubscribed(),
        emptyList()
    )

    init {
        fetchData()
    }

    private fun fetchData() = viewModelScope.launch {
        val response = repository.fetchData()
        delay(2000L)
        _meals.value = handleMealResponse(response)
    }

    private fun handleMealResponse(response: Response<MealResponse>)
            : Resource<MealResponse> {
        if (response.isSuccessful)
            response.body()?.let { return Resource.Success(it) }
        return Resource.Error(response.message())
    }

    fun fetchById(id: String) = viewModelScope.launch {
        val response = repository.fetchById(id)
        delay(2000L)
        _meal.value = handleMealResponse(response)
    }

    fun insert(meal: Meal) = viewModelScope.launch {
        repository.insert(meal)
    }

    fun delete(meal: Meal) = viewModelScope.launch {
        repository.delete(meal)
    }

    fun checkIfExistInLocalDb(meal: Meal): Boolean = meal in localMeals.value
}

## Interfejs użytkownika

Wykorzystamy `BottomNavigation`, na którym umieścimy dwa ekrany - wyświetlający listę pobraną z serwera, oraz listę pobraną z lokalnej bazy. Dodamy również trzeci ekran (poza `BottomNavigation`) wyświetlający widok szczegółowy.

W tym celu zaimplementujmy dwie klasy, pierwsza będzie reprezentowała wszystkie ekrany, druga tylko znajdujące się w nawigacji dolnej.

In [ ]:
sealed class Screens(
    val route: String
) {
    object Meals : Screens("meals")
    object Favorite : Screens("favorite")
    object Details : Screens("detail")
}

In [ ]:
sealed class BottomBar(
    val route: String,
    val title: String,
    val icon: ImageVector
) {
    object Home : BottomBar(Screens.Meals.route, "Meals", Icons.Default.Home)
    object Favorite : BottomBar(Screens.Favorite.route, "Favorite", Icons.Default.Favorite)
}

Dodajmy nawigację

In [ ]:
@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun Navigation() {
    val navController = rememberNavController()
    val viewModel: FoodViewModel = viewModel(
        LocalViewModelStoreOwner.current!!,
        "MealViewModel",
        FoodViewModelFactory(LocalContext.current.applicationContext as Application)
    )

    Scaffold(
        bottomBar = { BottomMenu(navController = navController)},
        content = { paddingValues ->
            BottomNavGraph(navController = navController, viewModel, paddingValues)
        }
    )
}

Głównym komponentem aplikacji będzie `Scaffold` zawierający `bottomBar`

In [ ]:
@Composable
fun BottomMenu(navController: NavHostController){
    val screens = listOf(
        BottomBar.Home, BottomBar.Favorite
    )

    val navBackStackEntry by navController.currentBackStackEntryAsState()
    val currentDestination = navBackStackEntry?.destination

    NavigationBar{
        screens.forEach{screen ->
            NavigationBarItem(
                label = { Text(text = screen.title)},
                icon = {Icon(imageVector = screen.icon, contentDescription = "icon")},
                selected = currentDestination?.hierarchy?.any { it.route == screen.route } == true,
                onClick = {navController.navigate(screen.route)}
            )
        }
    }
}

`BottomMenu` zawiera dwa ekrany

In [ ]:
@Composable
fun BottomNavGraph(navController: NavHostController, viewModel: FoodViewModel, paddingValues: PaddingValues){
    NavHost(navController = navController, startDestination = Screens.Meals.route) {
        composable(route = Screens.Meals.route){
            MealsScreen (
                navController = navController,
                viewModel = viewModel,
                paddingValues = paddingValues
            )
        }

        composable(route = Screens.Favorite.route){
            FavoriteScreen(
                navController = navController,
                viewModel = viewModel,
                paddingValues = paddingValues
            )
        }

        composable(route = Screens.Details.route){
            DetailScreen(
                viewModel = viewModel,
                paddingValues = paddingValues
            )
        }
    }
}

Do każdego komponentu reprezentującego ekran przekazujemy `viewModel` (współdzielenie) oraz `paddingValues` (aby odpowiednio ustawić marginesy i treść nie pokrywała się z `bottomBar`). Do ekranów zawierających listy przekazujemy również `navController`, aby móc dodać kod umożliwiający przejście na komponent reprezentujący ekran szczególu.

Zwróćmy uwagę na ekran pokazujący szczegóły dania, wyświetla on dane pobrane z serwera na podstawie przekazanego identyfikatora. Musielibyśmy zatem przekazać ten identyfikator z komponentu reprezentuującego ekrany list, do komponentu reprezentującego ekran szczegółu.

W tym przykładzie nie będziemy tego robić, zamiast tego zastosujemy **współdzielony** `ViewModel`. Dzięki temu, jeszcze znajdując się na ekranie listy, możemy wywołać metodę `fetchDataById`, która aktualizuje pole `meal`. Po przejściu na ekran szczególu, dodamy obserwatora do pola `meal` zawierające aktualne dane.

Na dwóch ekranach chcemy pokazać wiadomość z możliwym błędem, oraz odpowiedni komponent oznaczający ładowanie damych. Zaimplementujmy odpowiednie funkcje.

In [ ]:
@Composable
fun ShowErrorMessage(message: String) {
    Column(
        modifier = Modifier.fillMaxSize(),
        verticalArrangement = Arrangement.Center,
        horizontalAlignment = Alignment.CenterHorizontally
    ) {
        Text(
            text = "An error has occurred\n\n$message",
            fontSize = 48.sp
        )
    }
}

@Composable
fun ShowLoadingBar() {
    Column(
        modifier = Modifier.fillMaxSize(),
        verticalArrangement = Arrangement.Center,
        horizontalAlignment = Alignment.CenterHorizontally
    ) {
        CircularProgressIndicator(
            modifier = Modifier.width(64.dp)
        )
    }
}

Dodajmy również funkcję `ShowList` pokazującą listę wszystkich dań (z serwera lub lokalnej bazy)

In [ ]:
@Composable
fun ShowList(
    meals: List<Meal>, // lista którą chcemy wyświetlić
    paddingValues: PaddingValues,
    navController: NavController,
    viewModel: FoodViewModel
) {
    LazyColumn(modifier = Modifier.padding(paddingValues)) {
        items(meals) { meal ->
            Column(
                modifier = Modifier.fillMaxSize(),
                verticalArrangement = Arrangement.Center,
                horizontalAlignment = Alignment.CenterHorizontally
            ) {
                Card( // tło
                    modifier = Modifier
                        .fillMaxWidth()
                        .padding(start = 4.dp, end = 4.dp)
                        .clickable { // dodanie klikalności elementu
                            viewModel.fetchById(meal.idMeal) // zamiast przekazania id, wykonujemy pobranie danych
                                                             // będą przechowywant we współdzielonym viewmodel
                            navController.navigate(Screens.Details.route) // przechodzimy na ekran szczególu
                        }
                ) {
                    AsyncImage( // z biblioteki Coil
                        model = meal.strMealThumb, // adres grafiki
                        contentDescription = null,
                        contentScale = ContentScale.Crop,
                        modifier = Modifier
                            .fillMaxWidth()
                            .height(200.dp)
                    )
                    Text(
                        text = meal.strMeal,
                        Modifier
                            .padding(4.dp)
                            .fillMaxWidth(),
                        textAlign = TextAlign.Center,
                        fontSize = 24.sp
                    )
                }
                Spacer(modifier = Modifier.padding(12.dp))
            }
        }
    }
}

Dodajmy same ekrany.

In [ ]:
@Composable
fun MealsScreen(navController: NavController, viewModel: FoodViewModel, paddingValues: PaddingValues) {
    
    val response by viewModel.meals.collectAsStateWithLifecycle()

    when (response) {
        is Resource.Success -> { response.data?.let { ShowList(meals = it.meals, paddingValues, navController, viewModel) } }
        is Resource.Error -> { response.message?.let { ShowErrorMessage(message = it) } }
        is Resource.Loading -> { ShowLoadingBar() }
    }
}

In [ ]:
@Composable
fun FavoriteScreen(navController: NavController, viewModel: FoodViewModel, paddingValues: PaddingValues){

    val meals by viewModel.localMeals.collectAsStateWithLifecycle()

    ShowList(
        meals = meals,
        paddingValues = paddingValues,
        navController = navController,
        viewModel = viewModel
    )
}

Ostatnim ekranem jest `DetailScreen`, rozpocznijmy od dodania kilku komponentów.

W zależności od tego czy wyświetlany element znajduje się w lokalnej bazie, chcemy umożliwić dodanie, lub usunięcie elementu.

In [ ]:
@Composable
private fun AddToFavoriteFAB(
    onClick: () -> Unit
) {
    FloatingActionButton(
        onClick = onClick,
        elevation = FloatingActionButtonDefaults.bottomAppBarFabElevation(10.dp),
        containerColor = Color(100, 220, 237),
        shape = FloatingActionButtonDefaults.smallShape
    ) {
        Icon(
            imageVector = Icons.Filled.Add,
            contentDescription = "Add item"
        )
    }
}

@Composable
private fun RemoveFromFavoriteFAB(
    onClick: () -> Unit
) {
    FloatingActionButton(
        onClick = onClick,
        elevation = FloatingActionButtonDefaults.bottomAppBarFabElevation(10.dp),
        containerColor = Color(100, 220, 237),
        shape = FloatingActionButtonDefaults.smallShape
    ) {
        Icon(
            imageVector = Icons.Filled.Remove,
            contentDescription = "Remove item"
        )
    }
}

Chcemy pokazać tytuł bezpośrednio na obrazie, więc dodamy również gradient, aby napis był widoczny.

In [ ]:
@Composable
private fun Gradient() {
    Box(
        modifier = Modifier
            .fillMaxSize()
            .background(
                Brush.verticalGradient(
                    colors = listOf(
                        Color.Transparent,
                        Color.Black
                    ),
                    startY = 450f
                )
            )
    )
}

@Composable
private fun MealImage(url: String) {
    AsyncImage( // z biblioteki Coil
        model = url,
        contentDescription = "",
        contentScale = ContentScale.Crop,
        modifier = Modifier.fillMaxWidth()
    )
}

In [ ]:
@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun ShowMeal(
    meal: Meal, // dane do wyświetlenia
    viewModel: FoodViewModel,
    paddingValues: PaddingValues,
    modifier: Modifier = Modifier
) {

    Scaffold( // zagnieżedżony Scaffold
        modifier = Modifier.padding(paddingValues),
        floatingActionButton = { // przycisk dodawania/usuwania elementu z bazy
            if (viewModel.checkIfExistInLocalDb(meal))
                RemoveFromFavoriteFAB { viewModel.delete(meal) }
            else
                AddToFavoriteFAB { viewModel.insert(meal) }
        },
    ) { scaffoldPaddingValues -> // musimy dodać kolejne marginesy aby FAB był widoczny w odpowiednim miejscu
        Card(
            modifier = modifier
                .fillMaxWidth()
                .padding(scaffoldPaddingValues)
                .verticalScroll(rememberScrollState()), // umożliwienie przewijania na ekranie
            shape = RoundedCornerShape(15.dp),
            elevation = CardDefaults.cardElevation(defaultElevation = 10.dp)
        ) {
            Box(
                modifier = Modifier
                    .height(200.dp)
                    .fillMaxWidth()
            ) {
                MealImage(meal.strMealThumb)
                Gradient()
                Text(
                    text = meal.strMeal,
                    modifier = Modifier
                        .align(Alignment.BottomStart)
                        .padding(12.dp),
                    fontSize = 24.sp,
                    style = TextStyle(color = Color.White),
                    fontWeight = FontWeight.Bold
                )
            }
            Column(
                modifier = Modifier
                    .fillMaxWidth()
            ) {
                Text(
                    text = meal.strCategory,
                    modifier = Modifier.fillMaxWidth(),
                    textAlign = TextAlign.Center,
                    fontSize = 24.sp,
                    fontWeight = FontWeight.Bold
                )
                Text(
                    text = meal.strInstructions,
                    modifier = Modifier.padding(start = 16.dp, top = 24.dp, end = 16.dp)
                )
            }
        }
    }
}

Dodajmy główny komponent do głównej aktywności

In [ ]:
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContent {
            FlavorFinderComposeTheme {
                // A surface container using the 'background' color from the theme
                Surface(
                    modifier = Modifier.fillMaxSize(),
                    color = MaterialTheme.colorScheme.background
                ) {
                    Navigation()
                }
            }
        }
    }
}

Możemy przetestować aplikację.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExZzlweXR6dzI1b3ZpNnk4MXo3cTJ6Yjd1NWx3ZG85ZGNzNGkyOHZobCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/NqBQ91k7zOVh5d39NK/giphy.gif" width="200" />